In [1]:
import cooltools
import bioframe
import cooler
import pandas as pd
from HMMC import HMMC
import fileprocessing
from importlib import reload  
import numpy as np
from cooltools import eigdecomp
from scipy import stats
import matplotlib.pyplot as plt
from natsort import natsorted

In [2]:
cooldir = '/pollard/data/chromatin_organization/rao2014/distiller-0.3.1_hg19/results'
fasta_file='/pollard/data/vertebrate_genomes/human/hg19/hg19/hg19.fa'

In [3]:
cools = fileprocessing.get_files(cooldir)

In [5]:
df = fileprocessing.cools_df(cools, resolution = 50000)

In [6]:
df

cell_line assembly                                               path  \
0      NHEK     hg19  /pollard/data/chromatin_organization/rao2014/d...   
1     IMR90     hg19  /pollard/data/chromatin_organization/rao2014/d...   
2      K562     hg19  /pollard/data/chromatin_organization/rao2014/d...   
3     HUVEC     hg19  /pollard/data/chromatin_organization/rao2014/d...   
4      HMEC     hg19  /pollard/data/chromatin_organization/rao2014/d...   
5      KBM7     hg19  /pollard/data/chromatin_organization/rao2014/d...   
6   GM12878     hg19  /pollard/data/chromatin_organization/rao2014/d...   

                                              cooler  
0  <Cooler "NHEK_inSitu_MboI.hg19.mapq_30.1000.mc...  
1  <Cooler "IMR90_inSitu_MboI.hg19.mapq_30.1000.m...  
2  <Cooler "K562_inSitu_MboI.hg19.mapq_30.1000.mc...  
3  <Cooler "HUVEC_inSitu_MboI.hg19.mapq_30.1000.m...  
4  <Cooler "HMEC_inSitu_MboI.hg19.mapq_30.1000.mc...  
5  <Cooler "KBM7_inSitu_MboI.hg19.mapq_30.1000.mc...  
6  <Cooler "GM12878_inSitu_MboI.hg19.mapq_30.1000...

In [7]:
gc_dict = fileprocessing.get_gc(df, fasta_file)

In [8]:
gc_dict['hg19'].head()

chrom   start     end  weight        GC
0  chr1       0   50000     NaN  0.484250
1  chr1   50000  100000     NaN  0.376740
2  chr1  100000  150000     NaN  0.429960
3  chr1  150000  200000     NaN  0.447314
4  chr1  200000  250000     NaN  0.410796

In [9]:
c = df.iloc[6].cooler

In [10]:
chromsizes = c.chromsizes
chromsizes.head()

name
chr1    249250621
chr2    243199373
chr3    198022430
chr4    191154276
chr5    180915260
Name: length, dtype: int32

In [11]:
midpoints = bioframe.fetch_centromeres('hg19')
midpoints.head()

chrom      start        end        mid
0   chr1  121500000  128900000  125200000
1  chr10   38000000   42300000   40150000
2  chr11   51600000   55700000   53650000
3  chr12   33300000   38200000   35750000
4  chr13   16300000   19500000   17900000

In [12]:
chromarms = bioframe.genomeops.make_chromarms(chromsizes, midpoints)
chromarms = chromarms.replace("chrMp", "chrZ")
chromarms.head()

chrom      start        end    name
0   chr1          0  125200000   chr1p
1   chr1  125200000  249250621   chr1q
2  chr10          0   40150000  chr10p
3  chr10   40150000  135534747  chr10q
4  chr11          0   53650000  chr11p

In [13]:
rows = [row for index, row in chromarms.iterrows()]

In [14]:
regions = [(chrom, start, end) for chrom, start, end, name in natsorted(rows, key=lambda x: x[-1])]

In [15]:
cis_val, cis_eig = eigdecomp.cooler_cis_eig(
                c, 
                gc_dict['hg19'], 
                regions = regions,
                n_eigs=3, 
                balance=True,
                ignore_diags = 3, clip_percentile=99.9)

In [16]:
def get_corr(masked_track):
    for chrom in masked_track.chrom.unique():
        mask = masked_track.chrom==chrom
        corrs = [stats.pearsonr(masked_track[mask]['E' + str(i)].values, masked_track[mask]['GC'].values)[0] for i in range(1, 4)]

        print(chrom, corrs)

In [17]:
masked_track = cis_eig[~cis_eig['E1'].isna()]
get_corr(masked_track)

chr1 [0.5760729020807314, 0.2065656475174492, 0.35675200562613346]
chr2 [0.5203116292159613, 0.4548183416885827, -0.08213577413185459]
chr3 [0.6569516002500658, 0.22284012960355257, 0.2761650068437038]
chr4 [0.46107248276059404, 0.2260910578435542, 0.26968338049495016]
chr5 [0.5535487059851355, 0.32108218216638007, 0.18209876227868876]
chr6 [0.4918259801107308, 0.04172003990163683, 0.14495112812273112]
chr7 [0.5696311907257, 0.371450230592364, 0.19905008602188992]
chr8 [0.4926266901622367, -0.017246587738823323, 0.2074389638056193]
chr9 [0.5773280530253413, 0.30902078033573976, 0.016708141041244396]
chr10 [0.35101010752114564, 0.5448906089595038, 0.25189881224600735]
chr11 [0.6709529686809441, 0.20027860739143152, 0.25389301491222177]
chr12 [0.6231250702200569, 0.32462970645372535, 0.23193275909517858]
chr13 [0.5198499603930166, -0.09443489802895046, 0.4353309403129022]
chr14 [0.6141183722338776, 0.09087185561211715, 0.14868067962359174]
chr15 [0.41070636624608564, 0.051079961180937945

In [18]:
cis_val2, cis_eig2 = eigdecomp.cooler_cis_eig(
                c, 
                gc_dict['hg19'], 
                regions = [(chrom, 0, c.chromsizes[chrom]) for chrom in c.chromnames],
                n_eigs=3, 
                balance=True,
                ignore_diags = 10, clip_percentile=99)

In [26]:
masked_track2 = cis_eig2[~cis_eig2['E1'].isna()]
get_corr(masked_track2)

chr1 [0.646636349315758, 0.38374587337614585, 0.11003335083158777]
chr2 [0.5154302713967027, 0.5458181937156702, 0.08910430969252721]
chr3 [0.6874865870465962, 0.33579977036207603, -0.08791164380217152]
chr4 [0.5290036516373862, 0.17653233608452426, 0.12231018479864564]
chr5 [0.5568835958214122, 0.22805135107002114, 0.41462089129527374]
chr6 [0.5938188519175681, -0.15263773657558377, 0.24239931154664462]
chr7 [0.5535198432450481, 0.5144218092437026, 0.1171751286056024]
chr8 [0.46425298388258857, 0.3961745402837187, 0.40647415565693534]
chr9 [0.6523419371690512, -0.10186710693551114, 0.33515998649171036]
chr10 [0.3372668426631102, 0.4903489004592187, 0.09580035720489108]
chr11 [0.6898128789791588, 0.13740935917690542, -0.09486585485479752]
chr12 [0.6374526453637082, 0.42224103057644286, 0.08715353739690017]
chr13 [0.5282201488990541, -0.0775828042829656, 0.4182138454395837]
chr14 [0.6151305440980643, 0.10576491033877118, 0.15411605830432545]
chr15 [0.41359915476244097, 0.073808453549947

In [27]:
plt.figure(figsize=(20, 3))
chrom = 'chr19'
y1 = masked_track[masked_track.chrom == chrom]['E1'].values
plt.plot(y1)
y2 = masked_track2[masked_track2.chrom == chrom]['E1'].values
plt.plot(y2)

In [28]:
reload(HMMC)

<module 'HMMC.HMMC' from '/pollard/home/lchumpitaz/Hi-C/testing/HMMC/HMMC.py'>

In [45]:
hmm_track = HMMC.auto_analyze(cis_eig2)

Fitting a model with 47 free scalar parameters with only 38 data points will result in a degenerate solution.


[ 2. -2.  2. ... -3.  2.  0.]


In [46]:
hmm_track

chrom     start       end    weight        GC  E1  E2  E3  binary  HMM2  \
0      chr1         0     50000       NaN  0.484250 NaN NaN NaN     NaN   NaN   
1      chr1     50000    100000       NaN  0.376740 NaN NaN NaN     NaN   NaN   
2      chr1    100000    150000       NaN  0.429960 NaN NaN NaN     NaN   NaN   
3      chr1    150000    200000       NaN  0.447314 NaN NaN NaN     NaN   NaN   
4      chr1    200000    250000       NaN  0.410796 NaN NaN NaN     NaN   NaN   
...     ...       ...       ...       ...       ...  ..  ..  ..     ...   ...   
61923  chrY  59200000  59250000       NaN  0.388820 NaN NaN NaN     NaN   NaN   
61924  chrY  59250000  59300000       NaN  0.398220 NaN NaN NaN     NaN   NaN   
61925  chrY  59300000  59350000       NaN  0.479240 NaN NaN NaN     NaN   NaN   
61926  chrY  59350000  59373566       NaN  0.563615 NaN NaN NaN     NaN   NaN   
61927  chrM         0     16571  0.000135  0.444874 NaN NaN NaN     NaN   NaN   

       HMM3  HMM4  HMM5  HMM6  HMM5_2  
0       NaN   NaN   NaN   NaN     NaN  
1       NaN   NaN   NaN   NaN     NaN  
2       NaN   NaN   NaN   NaN     NaN  
3       NaN   NaN   NaN   NaN     NaN  
4       NaN   NaN   NaN   NaN     NaN  
...     ...   ...   ...   ...     ...  
61923   NaN   NaN   NaN   NaN     NaN  
61924   NaN   NaN   NaN   NaN     NaN  
61925   NaN   NaN   NaN   NaN     NaN  
61926   NaN   NaN   NaN   NaN     NaN  
61927   NaN   NaN   NaN   NaN     NaN  

[61928 rows x 15 columns]

In [47]:
plt.figure(figsize=(20, 3))
chrom = 'chr19'
mask = (~hmm_track['E1'].isna()) & (hmm_track.chrom == chrom)
y1 = hmm_track[mask]['E1'].values
plt.plot(y1)
y2 = hmm_track[mask]['HMM3'].values - 1
plt.plot(y2)

In [48]:
hmm_track.to_csv('../annotations/GM12878.csv', index=False)